In [ ]:
%%time

import glob

import geopandas as gpd
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
from libpysal.graph import read_parquet
from sklearn.preprocessing import PowerTransformer, RobustScaler, StandardScaler

from fast_hdbscan.numba_kdtree import kdtree_to_numba
from fast_hdbscan.numba_kdtree import parallel_tree_query
from sklearn.neighbors import KDTree

In [ ]:
region_id = 69300

tessellations_dir = '/data/uscuni-ulce/processed_data/tessellations/'
chars_dir = "/data/uscuni-ulce/processed_data/chars/"
graph_dir = "/data/uscuni-ulce/processed_data/neigh_graphs/"

In [ ]:
graph = read_parquet(graph_dir + f"tessellation_graph_{region_id}_knn1.parquet")

In [ ]:
graph.cardinalities.describe()

In [ ]:
# graph2 = graph.higher_order(lower_order=True, k=2, diagonal=True)
# graph2.cardinalities.describe()

In [ ]:
# graph3 = graph.higher_order(lower_order=True, k=3, diagonal=True)
# graph3

In [ ]:
from core.cluster_validation import print_distance, generate_neigbhourhood_groups

In [ ]:
tessellation = gpd.read_parquet(
        tessellations_dir + f"tessellation_{region_id}.parquet"
)

In [ ]:
X_train = pd.read_parquet(chars_dir + f'primary_chars_{region_id}.parquet')



X_train = X_train[X_train.index >= 0]



spatial_lag = 3


# lag = pd.read_parquet(f'/data/uscuni-ulce/processed_data/context_data/context_chars_{region_id}_lag_{spatial_lag}.parquet')

lag = pd.read_parquet(f'/data/uscuni-ulce/processed_data/context_data/unprocessed_context_chars_{region_id}_lag_{spatial_lag}.parquet')



# X_train = X_train.join(lag[[c for c in lag.columns if '_median' in c]], how='inner')
X_train = X_train.join(lag, how='inner')


In [ ]:
vals = StandardScaler().fit_transform(X_train)
X_train = pd.DataFrame(vals, columns=X_train.columns, index=X_train.index)

vals = np.nan_to_num(X_train)
X_train = pd.DataFrame(vals, columns=X_train.columns, index=X_train.index)


# X_train = X_train.clip(-10, 10)

In [ ]:
X_train.shape

In [ ]:
# X_train = pd.read_parquet('../data/old_prague_data/old_prague_data.parquet')
# X_train = X_train.set_index('uID').sort_index()


# vals = np.nan_to_num(StandardScaler().fit_transform(X_train))
# X_train = pd.DataFrame(vals, columns=X_train.columns, index=X_train.index)
# # X_train = X_train.clip(-10, 10)



In [ ]:
# tessellation = gpd.read_file('../data/old_prague_data/prg_geometry.gpkg', 
#                              layer='tessellation').set_index('uID').sort_index().to_crs(epsg=3035)
# from libpysal.graph import Graph
# graph = Graph.build_fuzzy_contiguity(tessellation, buffer=.001)

In [ ]:
tess_groups = generate_neigbhourhood_groups(tessellation[tessellation.index.isin(X_train.index)])
tess_groups = tess_groups[tess_groups.index.isin(X_train.index)]
tess_groups_ilocs = (
    pd.Series(np.arange(len(X_train)), index=X_train.index)
    .loc[tess_groups.index]
    .values
)

In [ ]:
neighbourhoods = X_train.loc[tess_groups.index].groupby(tess_groups.values).mean()
print_distance(neighbourhoods, metric='euclidean')

In [ ]:
from scipy.spatial.distance import pdist

In [ ]:
for i, g in X_train.loc[tess_groups.index].groupby(tess_groups.values):
    print(i, np.median(pdist(g)))

In [ ]:
# tessellation.loc[tess_groups.index].explore(column=tess_groups.values, categorical=True)

In [ ]:
newgraph = graph.subgraph(X_train.index.values)

In [ ]:
newgraph.component_labels.value_counts()

In [ ]:
main_comp = 0

In [ ]:
import lonboard
from sidecar import Sidecar
from libpysal.graph import Graph
from lonboard.colormap import apply_continuous_cmap
from core.cluster_validation import get_color

In [ ]:
plotting = tessellation.loc[newgraph.component_labels[newgraph.component_labels == main_comp].index]

In [ ]:
%%time
layer = lonboard.PolygonLayer.from_geopandas(plotting, opacity=.15)

Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [ ]:


sc = Sidecar(title='buildings')

Create a Map object

In [ ]:
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.Positron)

Display map within the sidecar plugin

In [ ]:
with sc:
    display(m)

In [ ]:
plot_data = X_train.loc[newgraph.component_labels[newgraph.component_labels == main_comp].index]
plot_graph = newgraph.higher_order(k=3, lower_order=True, diagonal=True).subgraph(plot_data.index)

In [ ]:
focals = plot_graph._adjacency.index.get_level_values(0).values
neighbours = plot_graph._adjacency.index.get_level_values(1).values
fvals = plot_data.loc[focals].values
nvals = plot_data.loc[neighbours].values

In [ ]:
distances = np.sqrt(np.sum(np.pow(fvals - nvals, 2), axis=1))

In [ ]:
# %%time
# from scipy.spatial.distance import cosine
# distances = [cosine(fvals[i], nvals[i]) for i in range(fvals.shape[0])]
# distances = np.array(distances)

In [ ]:
pd.Series(distances).describe().iloc[1:]

In [ ]:
threshold = 8


In [ ]:
new_focals = focals[np.where(distances <= threshold)]
new_neighbours = neighbours[np.where(distances <= threshold)]
new_distances = distances[np.where(distances <= threshold)]

subgraph = Graph.from_arrays(new_focals, new_neighbours, new_distances)

In [ ]:
# tessellation.loc[subgraph.component_labels[subgraph.component_labels == 78865].index].explore()

In [ ]:
ntop = 100

top_components = subgraph.component_labels.value_counts()

labels = subgraph.component_labels.copy()
top_labels = top_components[:ntop].index.values
labels[~labels.isin(top_labels)] = -1
repl_dict = dict(zip(top_labels.tolist(), np.arange(ntop).tolist()))
labels = labels.replace(repl_dict)

In [ ]:
layer.get_fill_color = get_color(labels)

### Hierarchical

In [ ]:
# embedding = X_train[[c for c in X_train.columns if '_median' in c]]
embedding = X_train

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import fcluster
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import davies_bouldin_score

In [ ]:
clustering_graph = graph.subgraph(X_train.index.values).transform('B').sparse

In [ ]:
clusterer = AgglomerativeClustering(linkage='ward',
                                    connectivity=clustering_graph, 
                                    compute_full_tree=True, compute_distances=True)

In [ ]:
%%time
model = clusterer.fit(embedding)

In [ ]:
def get_linkage_matrix(model):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)
    return linkage_matrix

In [ ]:
linkage_matrix = get_linkage_matrix(model)

In [ ]:
fix, ax = plt.subplots(figsize=(40,40))
# Plot the corresponding dendrogram
_ = dendrogram(linkage_matrix, truncate_mode="level", p=5, ax=ax)

In [ ]:
%%time
import lonboard
plotting = tessellation[tessellation.index.isin(X_train.index)].copy()
layer = lonboard.PolygonLayer.from_geopandas(plotting, opacity=.15)

In [ ]:
from sidecar import Sidecar
sc = Sidecar(title='Clusters')
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.Positron)
with sc:
    display(m)

In [ ]:
from core.cluster_validation import get_color

In [ ]:
for t in range(100, 1250, 100):
    r = fcluster(linkage_matrix, t=t, criterion='distance')
    # r = pd.Series(r, index=X_train.index)
    # # ssplits = graph.describe(r, statistics=['nunique'])['nunique']
    print(t, ' - ', 
          adjusted_rand_score(tess_groups.values, r[tess_groups_ilocs]),
          # (ssplits > 1).sum() / ssplits.shape[0],
          davies_bouldin_score(X_train, r)
         )

In [ ]:
clusters = fcluster(linkage_matrix, t=350, criterion='distance')

In [ ]:
np.unique(clusters, return_counts=True)

In [ ]:
layer.get_fill_color = get_color(clusters)

In [ ]:
sklearn_tree = KDTree(X_train.values)
numba_tree = kdtree_to_numba(sklearn_tree)

In [ ]:
from core.cluster_validation import generate_neigbhourhood_groups

In [ ]:
groups = generate_neigbhourhood_groups(tessellation)

In [ ]:
karlins = groups[groups == 'karlin'].index.values

In [ ]:
tess_id = 261793

In [ ]:
# tessellation.loc[graph[tess_id].index.values].explore()

In [ ]:
graph[tess_id].index.values

In [ ]:
xilocs = np.where(X_train.index.isin(graph[tess_id].index.values))[0]

In [ ]:
num_neighbours = 15

In [ ]:
nn_dists, nn_inds = parallel_tree_query(numba_tree, np.ascontiguousarray(X_train.iloc[xilocs].values), k=num_neighbours)

In [ ]:
neigh_ids = nn_inds[3]
neigh_dists = nn_dists[3]

In [ ]:
neighbours = tessellation.loc[X_train.iloc[neigh_ids].index].reset_index()
neighbours['nn_dists'] = neigh_dists
neighbours['nn_order'] = np.arange(num_neighbours)

In [ ]:
intersection = np.intersect1d(X_train.iloc[neigh_ids].index, graph[tess_id].index.values)
union = np.union1d(X_train.iloc[neigh_ids].index, graph[tess_id].index.values)

print('graph intersection: ', intersection.shape[0] / graph[tess_id].index.values.shape[0],
      'total intersection: ', intersection.shape[0]/union.shape[0])

In [ ]:
from scipy.spatial.distance import pdist, cdist

In [ ]:
pdist(X_train.iloc[neigh_ids].values)

In [ ]:
# m = neighbours.explore()
# m = tessellation.loc[[tess_id]].explore(color='red', m=m)
# m